In [1]:
ls

CoronaNet/                          README.md
Get_Chinese_Subnational_Data.ipynb  SelfAnalysis.ipynb


In [2]:
ls ../data

OxCGRT_US_states_temp.csv     OxCGRT_latest_responses.csv
OxCGRT_latest.csv             OxCGRT_latest_withnotes.csv
OxCGRT_latest_allchanges.csv  timeseries/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)

#### First, we extra original data from `OxCGRT_latest_withnotes`

In [5]:
df_latest_withnotes = pd.read_csv('../data/OxCGRT_latest.csv')
df_latest_withnotes.head(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Great, so now we need to examine the data from CoronaNet and find the corresponding columns / policy code. 

In [6]:
ls CoronaNet/

README.md                      s41562-020-0909-7.pdf
coronanet_release_allvars.csv


In [7]:
df_corona_net = pd.read_csv("./CoronaNet/coronanet_release_allvars.csv")

In [8]:
# for now, we only want the chinese data
df_corona_net_china = df_corona_net[df_corona_net.country == 'China']
df_corona_net_china.head(2)

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
3,R_3EQBDJGMTKwowKOAq,2272584,new_entry,correction,NaN,NaN,China announced on January 22: to guide the pr...,2020-01-22,2020-01-22,NaN,China,CHN,CN,National,0,NaN,NaN,Public Awareness Measures,Disseminating information related to COVID-19 ...,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.655965,51.59842,50.520158,194.0,http://wjw.wuhan.gov.cn/ztzl_28/fk/tzgg/202004...,2020-07-20,2020-07-20T21:13:39Z
4,R_1OoGDBSpbZQ02oANA,5468900,new_entry,original,NaN,NaN,"Xi'an, China announced that subway stations an...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,0,Shaanxi,Xi'an,Hygiene,NaN,NaN,NaN,China,NaN,NaN,Shaanxi,Xi'an,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Municipal Government,52.655965,51.59842,50.520158,194.0,http://www.sxdaily.com.cn/2020-01/30/content_8...,2020-07-16,2020-07-16T07:39:56Z


In [9]:
# and we only want province-level data
df_corona_net_china = df_corona_net_china[df_corona_net_china.province.notna()]

In [10]:
# Also, it only makes sense to record policies with a clear start and end date for now
df_corona_net_china = df_corona_net_china[(df_corona_net_china.date_end.notna() & 
                                           df_corona_net_china.date_start.notna())]
df_corona_net_china.head(2)

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
81,R_27eQUD05p74fORiNA,3484811,update,original,End of Policy,NaN,Guizhou raised health emergency response to th...,2020-01-24,2020-01-24,2020-02-24,China,CHN,CN,Provincial,0,Guizhou,NaN,Declaration of Emergency,NaN,NaN,NaN,China,NaN,NaN,Guizhou,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Provincial/State Government,53.063301,51.961999,50.848488,194.0,http://china.caixin.com/2020-01-29/101509411.html,2020-07-02,2020-07-02T02:01:25Z
82,R_oYvDmnyoZKwO4uJNA,3910940,update,original,End of Policy,NaN,Yunnan raised health emergency response to the...,2020-01-24,2020-01-24,2020-02-24,China,CHN,CN,Provincial,0,Yunnan,NaN,Declaration of Emergency,NaN,NaN,NaN,China,NaN,NaN,Yunnan,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Provincial/State Government,53.063301,51.961999,50.848488,194.0,http://china.caixin.com/2020-01-29/101509411.html,2020-07-02,2020-07-02T01:59:04Z


#### Let's compare the columns and figure out what we need

The meaning of codes can be found [here](https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md) (oxford) and [here](./CoronaNet_Codebook.pdf) (coronanet)

In [11]:
df_latest_withnotes.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag',
       'C3_Cancel public events', 'C3_Flag', 'C4_Restrictions on gatherings',
       'C4_Flag', 'C5_Close public transport', 'C5_Flag',
       'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealth

In [12]:
df_corona_net_china.columns

Index(['record_id', 'policy_id', 'entry_type', 'correct_type', 'update_type',
       'update_level', 'description', 'date_announced', 'date_start',
       'date_end', 'country', 'ISO_A3', 'ISO_A2', 'init_country_level',
       'domestic_policy', 'province', 'city', 'type', 'type_sub_cat',
       'type_text', 'school_status', 'target_country', 'target_geog_level',
       'target_region', 'target_province', 'target_city', 'target_other',
       'target_who_what', 'target_direction', 'travel_mechanism', 'compliance',
       'enforcer', 'index_high_est', 'index_med_est', 'index_low_est',
       'index_country_rank', 'link', 'date_updated', 'recorded_date'],
      dtype='object')

To see how they correspond to each other, check out [README.md](./README.md) in this directory.

For now, we only want province-level data

In [13]:
df = df_corona_net_china[['country', 'ISO_A3', 'province', 'date_start', 'date_end', 'type', 'type_sub_cat', 'travel_mechanism', 'compliance']]
df.head()

,country,ISO_A3,province,date_start,date_end,type,type_sub_cat,travel_mechanism,compliance
81,China,CHN,Guizhou,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,Mandatory (Unspecified/Implied)
82,China,CHN,Yunnan,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,Mandatory (Unspecified/Implied)
83,China,CHN,Jiangsu,2020-01-24,2020-02-20,Restriction and Regulation of Government Services,Parks,NaN,Mandatory (Unspecified/Implied)
84,China,CHN,Jiangsu,2020-01-24,2020-02-20,Restriction and Regulation of Government Services,Tourist Sites,NaN,Mandatory (Unspecified/Implied)
85,China,CHN,Jiangsu,2020-01-24,2020-02-20,Restriction and Regulation of Government Services,Public libraries,NaN,Mandatory (Unspecified/Implied)


In [14]:
province_code = {
 'Anhui': 'AH',
 'Hong Kong': 'HK',
 'Shandong': 'SD',
 'Beijing': 'BJ',
 'Heilongjiang': 'HL',
 'Shanghai': 'SH',
 'Chongqing': 'CQ',
 'Hunan': 'HN',
 'Shaanxi': 'SN',
 'Fujian': 'FJ',
 'Jilin': 'JL',
 'Shanxi': 'SX',
 'Guangdong': 'GD',
 'Jiangsu': 'JS',
 'Tianjin': 'TJ',
 'Gansu': 'GS',
 'Jiangxi': 'JX',
 'Guangxi': 'GX',
 'Liaoning': 'LN',
 'Xinjiang': 'XJ',
 'Guizhou': 'GZ',
 'Macau': 'MO',
 'Tibet': 'XZ',
 'Henan': 'HA',
 'Inner Mongolia': 'NM',
 'Yunnan': 'YN',
 'Hubei': 'HB',
 'Zhejiang': 'ZJ',
 'Hebei': 'HE',
 'Qinghai': 'QH',
 'Hainan': 'HI',
 'Sichuan': 'SC',
 'Ningxia Hui Autonomous Region': 'NX'
}

In [15]:
df_latest_withnotes.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag',
       'C3_Cancel public events', 'C3_Flag', 'C4_Restrictions on gatherings',
       'C4_Flag', 'C5_Close public transport', 'C5_Flag',
       'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealth

In [16]:
region_code_series = []
for n in df_corona_net_china.province:
    region_code_series.append(province_code[n])
    
region_code_series = pd.Series(region_code_series)
df.insert(3, "region_code", region_code_series)
df.head(3)

,country,ISO_A3,province,region_code,date_start,date_end,type,type_sub_cat,travel_mechanism,compliance
81,China,CHN,Guizhou,HK,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,Mandatory (Unspecified/Implied)
82,China,CHN,Yunnan,HK,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,Mandatory (Unspecified/Implied)
83,China,CHN,Jiangsu,HK,2020-01-24,2020-02-20,Restriction and Regulation of Government Services,Parks,NaN,Mandatory (Unspecified/Implied)
